In [1]:
from google.colab import drive
import os
drive.mount('/content/gdrive')
os.chdir("gdrive/MyDrive/Yin/landscape")

Mounted at /content/gdrive


In [2]:
!pip install pyshp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 kB 3.9 MB/s eta 0:00:00


In [ ]:
!git clone https://github.com/VeryLargeGraph/TSCAN.git
!cd tSCAN
!pip install networkx
!pip install click

fatal: destination path 'TSCAN' already exists and is not an empty directory.
/bin/bash: line 0: cd: tSCAN: No such file or directory
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
def get_dict():
	mFile = open("analysis/data/forest_landscape_restoration_country_tab.tsv", "r",encoding="cp437", errors='ignore')
	next(mFile)
	countries = {}
	for ele in mFile:
		data = ele.rstrip().split('\t')
		country = data[-3]
		coord = data[-2]
		author = data[-33]
		if author in countries:
			pass
		else:
			countries[author] = [country,coord]
	return countries

In [16]:
import pandas as pd
import networkx as nx
import numpy as np
from ast import literal_eval
import matplotlib.pyplot as plt

In [5]:
!pip install netgraph

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.1/96.1 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.8/300.8 kB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 4.4 MB/s eta 0:00:00


In [6]:
!pip install igraph

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 64.5 MB/s eta 0:00:00


In [20]:
from community import community_louvain
import networkx as nx
# Import libraries
import pandas as pd #For reading dataset files
import networkx as nx #For network creation/analysis
from networkx.algorithms import community
import community as community_louvain
import matplotlib.pyplot as plt #For plotting graphs
import igraph as ig
import networkx.algorithms.community as nxcom

G = nx.Graph()
# G = nx.DiGraph()


df = pd.read_csv("analysis/data/forest_landscape_restoration_country_tab.tsv", sep ='\t')
df['retweeted_username'].replace('', np.nan, inplace=True)
df.dropna(subset=['retweeted_username'], inplace=True)
df['entities.mentions'].replace('', np.nan, inplace=True)
df.dropna(subset=['entities.mentions'], inplace=True)
df['entities.mentions'] = df['entities.mentions'].apply(literal_eval)
# print(df.head())
xx = get_dict()
for index, row in df.iterrows():
  orgNode = row["retweeted_username"]
  if orgNode in G.nodes():
    pass
  else:
    if orgNode in xx:
      G.add_node(orgNode, country = xx[orgNode])
      # G.add_node(orgNode)
  data = row["entities.mentions"]
  for ele in data:
    destNode = str(ele).replace("@", "")
    if destNode in G.nodes():
      G.add_edge(orgNode, destNode)
    else:
      if destNode in xx:
        # G.add_node(destNode)
        G.add_node(destNode, country = xx[destNode])
        G.add_edge(orgNode,destNode)
print(G.edges()) 
    # print(row["retweeted_username"], row["entities.mentions"])
remove = [node for node,degree in dict(G.degree()).items() if degree < 3]
G.remove_nodes_from(remove)
G.remove_edges_from(nx.selfloop_edges(G))

communities = sorted(nxcom.greedy_modularity_communities(G), key=len, reverse=True)


<ipython-input-20-f8f15b8b8933>:16: DtypeWarning: Columns (29,31,32,33,39,54,55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("analysis/data/forest_landscape_restoration_country_tab.tsv", sep ='\t')


[('ICRAF', 'CIFOR'), ('ICRAF', 'YIL_Initiative'), ('ICRAF', 'GovernorMwadime'), ('ICRAF', 'TaitaTaveta006'), ('ICRAF', 'UKinKenya'), ('ICRAF', 'Environment_Ke'), ('ICRAF', 'WangariCM'), ('ICRAF', 'IlianaMonterros'), ('ICRAF', 'FTA_Partnership'), ('ICRAF', 'marynjeng'), ('ICRAF', 'UNCCDLibrary'), ('ICRAF', 'ManuelGuarigua1'), ('ICRAF', 'Yale_ELTI'), ('ICRAF', 'DENROfficial'), ('ICRAF', 'AfriClimatConvo'), ('ICRAF', 'DrylandsTransf1'), ('ICRAF', 'lawinowiecki'), ('ICRAF', 'Focali_se'), ('ICRAF', 'EvergreeningA'), ('ICRAF', 'RegreenAfrica'), ('ICRAF', 'andyheald'), ('ICRAF', 'GlobalLF'), ('ICRAF', 'EltsonF'), ('ICRAF', 'VickiWangui'), ('ICRAF', 'GPFLRtweets'), ('ICRAF', 'SundayGeofrey5'), ('ICRAF', 'GlobalLF_Esp'), ('ICRAF', 'CIFOR_bosques'), ('ICRAF', 'UNBiodiversity'), ('ICRAF', 'FAOForestry'), ('ICRAF', 'WWFLeadForests'), ('ICRAF', 'IUCN'), ('ICRAF', 'IUFRO'), ('ICRAF', 'theGEF'), ('ICRAF', 'KeForestService'), ('ICRAF', 'FAOKenya'), ('ICRAF', 'WorldResources'), ('ICRAF', 'WLE_CGIAR'), 

In [18]:
import networkx as nx
from matplotlib import pyplot as plt
plt.rcParams.update(plt.rcParamsDefault)
plt.rcParams.update({'figure.figsize': (15, 10)})
# get reproducible results
import networkx.algorithms.community as nxcom

import random
from numpy import random as nprand
random.seed(123)
nprand.seed(123)

G = nx.Graph()
# G = nx.DiGraph()


df = pd.read_csv("analysis/data/forest_landscape_restoration_country_tab.tsv", sep ='\t')
# df = pd.read_csv("analysis/data/landscape_restoration_country_tab.tsv", sep ='\t')
# df = pd.read_csv("analysis/data/ecosystem_restoration_country_tab.tsv", sep ='\t')
# df = pd.read_csv("analysis/data/ecological_restoration_country_tab.tsv", sep ='\t')

df['retweeted_username'].replace('', np.nan, inplace=True)
df.dropna(subset=['retweeted_username'], inplace=True)
df['entities.mentions'].replace('', np.nan, inplace=True)
df.dropna(subset=['entities.mentions'], inplace=True)
df['entities.mentions'] = df['entities.mentions'].apply(literal_eval)
# print(df.head())
countries = get_dict()
for index, row in df.iterrows():
  orgNode = row["retweeted_username"]
  mcoord = row["coord"]
  mym = row["year_month"]
  myear = row["year"]
  if orgNode in G.nodes():
    nx.set_node_attributes(G, { orgNode:{'coord': mcoord, 'ym':mym, 'year': myear}})
  else:
    #'person1', name='John Doe', age=40
    G.add_node(orgNode, coord = mcoord, ym = mym, year = myear)
  data = row["entities.mentions"]
  for ele in data:
    destNode = str(ele).replace("@", "")
    if destNode in G.nodes():
      G.add_edge(orgNode, destNode)
    else:
      G.add_node(destNode, ym = mym, year = myear)
      G.add_edge(orgNode,destNode)
print(G.edges()) 
    # print(row["retweeted_username"], row["entities.mentions"])
remove = [node for node,degree in dict(G.degree()).items() if degree < 3]
G.remove_nodes_from(remove)
G.remove_edges_from(nx.selfloop_edges(G))

communities = sorted(nxcom.greedy_modularity_communities(G), key=len, reverse=True)
    # Count the communities
print(f"The karate club has {len(communities)} communities.")
# for ele in communities:
#   print(len(ele))


<ipython-input-18-e5a5ba778191>:17: DtypeWarning: Columns (29,31,32,33,39,54,55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("analysis/data/forest_landscape_restoration_country_tab.tsv", sep ='\t')


[('ICRAF', 'CIFOR'), ('ICRAF', 'YIL_Initiative'), ('ICRAF', 'GovernorMwadime'), ('ICRAF', 'TaitaTaveta006'), ('ICRAF', 'UKinKenya'), ('ICRAF', 'Environment_Ke'), ('ICRAF', 'FTA_Partnership'), ('ICRAF', 'WangariCw'), ('ICRAF', 'BourneMieke'), ('ICRAF', 'Njoki_Carol83'), ('ICRAF', 'WangariCM'), ('ICRAF', 'IlianaMonterros'), ('ICRAF', 'marynjeng'), ('ICRAF', 'UNCCDLibrary'), ('ICRAF', 'ManuelGuarigua1'), ('ICRAF', 'Yale_ELTI'), ('ICRAF', 'DENROfficial'), ('ICRAF', 'AfriClimatConvo'), ('ICRAF', 'DrylandsTransf1'), ('ICRAF', 'lawinowiecki'), ('ICRAF', 'Focali_se'), ('ICRAF', 'EvergreeningA'), ('ICRAF', 'RegreenAfrica'), ('ICRAF', 'andyheald'), ('ICRAF', 'GlobalLF'), ('ICRAF', 'EltsonF'), ('ICRAF', 'VickiWangui'), ('ICRAF', 'GPFLRtweets'), ('ICRAF', 'SundayGeofrey5'), ('ICRAF', 'GlobalLF_Esp'), ('ICRAF', 'CIFOR_bosques'), ('ICRAF', 'CITES'), ('ICRAF', 'UNBiodiversity'), ('ICRAF', 'FAOForestry'), ('ICRAF', 'WWFLeadForests'), ('ICRAF', 'IUCN'), ('ICRAF', 'WorldBank'), ('ICRAF', 'IUFRO'), ('ICR

In [11]:
def set_node_community(G, communities):
  for c, v_c in enumerate(communities):
    for v in v_c:
      G.nodes[v]['community'] = c + 1
def set_edge_community(G):
  for v, w, in G.edges:
    if G.nodes[v]['community'] == G.nodes[w]['community']:
      G.edges[v, w]['community'] = G.nodes[v]['community']
    else:
      G.edges[v, w]['community'] = 0
def get_color(i, r_off=1, g_off=1, b_off=1):
  r0, g0, b0 = 0, 0, 0
  n = 16
  low, high = 0.1, 0.9
  span = high - low
  r = low + span * (((i + r_off) * 3) % n) / (n - 1)
  g = low + span * (((i + g_off) * 5) % n) / (n - 1)
  b = low + span * (((i + b_off) * 7) % n) / (n - 1)
  return (r, g, b)

In [12]:
# Set node and edge communities
set_node_community(G, communities)
set_edge_community(G)
node_color = [get_color(G.nodes[v]['community']) for v in G.nodes]
# Set community color for edges between members of the same community (internal) and intra-community edges (external)
# external = [(v, w) for v, w in G.edges if G.edges[v, w]['community'] == 0]
# internal = [(v, w) for v, w in G.edges if G.edges[v, w]['community'] > 0]

external = [(v, w) for v, w in G.edges if G.edges[v, w]['community'] > 6]
internal = [(v, w) for v, w in G.edges if G.edges[v, w]['community'] < 7]

internal_color = ['black' for e in internal]

In [8]:
###This is the code that restrict nodes with country informaiton only
import networkx as nx
from matplotlib import pyplot as plt
plt.rcParams.update(plt.rcParamsDefault)
plt.rcParams.update({'figure.figsize': (15, 10)})
# get reproducible results
import networkx.algorithms.community as nxcom

import random
from numpy import random as nprand
random.seed(123)
nprand.seed(123)

G = nx.Graph()
# G = nx.DiGraph()


df = pd.read_csv("analysis/data/forest_landscape_restoration_country_tab.tsv", sep ='\t')
# df = pd.read_csv("analysis/data/landscape_restoration_country_tab.tsv", sep ='\t')
# df = pd.read_csv("analysis/data/ecosystem_restoration_country_tab.tsv", sep ='\t')
# df = pd.read_csv("analysis/data/ecological_restoration_country_tab.tsv", sep ='\t')

df['retweeted_username'].replace('', np.nan, inplace=True)
df.dropna(subset=['retweeted_username'], inplace=True)
df['entities.mentions'].replace('', np.nan, inplace=True)
df.dropna(subset=['entities.mentions'], inplace=True)
df['entities.mentions'] = df['entities.mentions'].apply(literal_eval)
# print(df.head())
countries = get_dict()
for index, row in df.iterrows():
  orgNode = row["retweeted_username"]
  mcoord = row["coord"]
  mym = row["year_month"]
  myear = row["year"]
  if orgNode in countries:
    if orgNode in G.nodes():
      nx.set_node_attributes(G, { orgNode:{'coord': mcoord, 'ym':mym, 'year': myear, 'country':countries[orgNode]}})
    else:
      G.add_node(orgNode, coord = mcoord, ym = mym, year = myear, country = countries[orgNode])
  else:
    pass
  data = row["entities.mentions"]
  for ele in data:
    destNode = str(ele).replace("@", "")
    if destNode in countries:
      if destNode in G.nodes():
        nx.set_node_attributes(G, { destNode:{'coord': countries[destNode][1], 'ym':mym, 'year': myear, 'country':countries[destNode]}})
        G.add_edge(orgNode, destNode)
      else:
        G.add_node(destNode, ym = mym, year = myear, country = countries[destNode], coord = countries[destNode][1])
        G.add_edge(orgNode,destNode)
print(G.edges()) 
    # print(row["retweeted_username"], row["entities.mentions"])
remove = [node for node,degree in dict(G.degree()).items() if degree < 3]
G.remove_nodes_from(remove)
G.remove_edges_from(nx.selfloop_edges(G))

communities = sorted(nxcom.greedy_modularity_communities(G), key=len, reverse=True)
    # Count the communities
print(f"The graph has {len(communities)} communities.")
# for ele in communities:
#   print(len(ele))

<ipython-input-8-2b56a78e837e>:18: DtypeWarning: Columns (29,31,32,33,39,54,55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("analysis/data/forest_landscape_restoration_country_tab.tsv", sep ='\t')


[('ICRAF', 'CIFOR'), ('ICRAF', 'YIL_Initiative'), ('ICRAF', 'GovernorMwadime'), ('ICRAF', 'TaitaTaveta006'), ('ICRAF', 'UKinKenya'), ('ICRAF', 'Environment_Ke'), ('ICRAF', 'WangariCM'), ('ICRAF', 'IlianaMonterros'), ('ICRAF', 'FTA_Partnership'), ('ICRAF', 'marynjeng'), ('ICRAF', 'UNCCDLibrary'), ('ICRAF', 'ManuelGuarigua1'), ('ICRAF', 'Yale_ELTI'), ('ICRAF', 'DENROfficial'), ('ICRAF', 'AfriClimatConvo'), ('ICRAF', 'DrylandsTransf1'), ('ICRAF', 'lawinowiecki'), ('ICRAF', 'Focali_se'), ('ICRAF', 'EvergreeningA'), ('ICRAF', 'RegreenAfrica'), ('ICRAF', 'andyheald'), ('ICRAF', 'GlobalLF'), ('ICRAF', 'EltsonF'), ('ICRAF', 'VickiWangui'), ('ICRAF', 'SundayGeofrey5'), ('ICRAF', 'GlobalLF_Esp'), ('ICRAF', 'CIFOR_bosques'), ('ICRAF', 'UNBiodiversity'), ('ICRAF', 'FAOForestry'), ('ICRAF', 'IUCN'), ('ICRAF', 'IUFRO'), ('ICRAF', 'theGEF'), ('ICRAF', 'KeForestService'), ('ICRAF', 'FAOKenya'), ('ICRAF', 'WorldResources'), ('ICRAF', 'WLE_CGIAR'), ('ICRAF', 'DavidKaimowitz'), ('ICRAF', 'jmollins'), ('I

In [9]:
## Create shapefile for each node with community information
for index, community in enumerate(communities):
  print(index)
  for ele in community:
    print(G.nodes[ele])

0
{'coord': '-123.11934,49.24966', 'ym': '2019-11', 'year': 2019, 'country': ['CA', '-113.64258000000001,60.10867']}
{}
{'ym': '2015-06', 'year': 2015, 'country': ['RW', '30.0,-2.0'], 'coord': '30.0,-2.0'}
{'ym': '2016-11', 'year': 2016, 'country': ['GB', '-3.1964799999999998,55.952059999999996'], 'coord': '-98.5,39.76'}
{'ym': '2015-03', 'year': 2015, 'country': ['KE', '38.0,1.0'], 'coord': '13.41053,52.52437'}
{'coord': '38.0,1.0', 'ym': '2018-02', 'year': 2018, 'country': ['US', '-77.00025,38.91706']}
{'coord': '-92.50044,34.750370000000004', 'ym': '2015-12', 'year': 2015, 'country': ['US', '-92.50044,34.750370000000004']}
{'coord': '13.333329999999998,47.33333', 'ym': '2020-12', 'year': 2020, 'country': ['CZ', '16.607960000000002,49.19522']}
{'ym': '2015-12', 'year': 2015, 'country': ['US', '-119.75126000000002,37.25022'], 'coord': '-119.75126000000002,37.25022'}
{'coord': '124.82666,11.081380000000001', 'ym': '2020-08', 'year': 2020, 'country': ['CH', '8.01427,47.00016']}
{}
{}
{'

In [ ]:
# for ele in G.nodes():
#   print(G.nodes[ele]['community'])
for ele in communities[0]:
  print(ele)
  print(G.nodes[ele])

# print(communities[0])
# print(communities[1])



DannyWijnhoud
{'coord': '4.88969,52.37403', 'ym': '2018-08', 'year': 2018, 'country': ['NL', '4.88969,52.37403']}
andersen_inger
{'ym': '2015-03', 'year': 2015, 'country': ['KE', '38.0,1.0'], 'coord': '13.41053,52.52437'}
IISD_ENB
{}
UNCCDLibrary
{'coord': '10.5,51.5', 'ym': '2018-08', 'year': 2018, 'country': ['DE', '10.5,51.5']}
Algorand
{'ym': '2021-03', 'year': 2021, 'country': ['US', '-71.05977,42.35843'], 'coord': '-71.05977,42.35843'}
GFIS_net
{}
farmradio
{'coord': '-113.64258000000001,60.10867', 'ym': '2016-08', 'year': 2016, 'country': ['CA', '-113.64258000000001,60.10867']}
theGCF
{'ym': '2020-07', 'year': 2020, 'country': ['KR', '127.75,36.5'], 'coord': '127.75,36.5'}
IUCN
{'coord': '8.01427,47.00016', 'ym': '2015-02', 'year': 2015, 'country': ['CH', '8.01427,47.00016']}
CWatsonICRAF
{'ym': '2019-02', 'year': 2019, 'country': ['KE', '38.0,1.0'], 'coord': '38.0,1.0'}
NGP_TechAssist
{}
IUCN_forests
{'ym': '2015-01', 'year': 2015, 'country': ['CH', '8.01427,47.00016'], 'coord'

In [ ]:
print(xx['TimChristo'])

['DK', '10.0,56.0']


In [21]:
from matplotlib.lines import Line2D
karate_pos = nx.spring_layout(G, k=.75)
plt.rcParams.update({'figure.figsize': (45, 30)})

legend_elements=[]
labels = {}    
xx_color =[]
for node in G.nodes():
  if G.nodes[node]['community'] < 2:
    labels[node] = node
    # print(G.nodes[node]['community'])
    # print(G.nodes[node]['community'])

# for ele in [1,2,3,4,5]:
for ele in [1]:
  xx_color = get_color(ele)
  legend_elements.append(Line2D([0], [0], marker='o', color='w', label='Community: ' + str(ele), markerfacecolor = xx_color, markersize=24))
    # Draw external edges
# nx.draw_networkx(
#         G,
#         pos=karate_pos,
#         with_labels=False,
#         node_color=node_color,
#         # node_size=0,
#         edgelist=external,
#         edge_color="silver",
#         style=":")
# nx.draw_networkx_labels(G, pos=karate_pos, labels=labels, font_size=16, font_color='r')

# #     # Draw nodes and internal edges
nx.draw_networkx(
        G,
        pos=karate_pos,
        node_size=150,
        with_labels=False,
        node_color=node_color,
        edgelist=internal,
        # edge_color=internal_color,
        alpha=0.5,
        edge_color="silver",
        style=":")
nx.draw_networkx_labels(G, pos=karate_pos, labels=labels, font_size=14)
# plt.legend()
plt.legend(handles=legend_elements, loc='upper right', fontsize = 18)
plt.show()

KeyError: ignored

In [13]:
from community import community_louvain
import networkx as nx
# Import libraries
import pandas as pd #For reading dataset files
import networkx as nx #For network creation/analysis
from networkx.algorithms import community
import community as community_louvain
import matplotlib.pyplot as plt #For plotting graphs
import igraph as ig
import networkx.algorithms.community as nxcom

G = nx.Graph()
# G = nx.DiGraph()

mFile = open("test.txt","w")
df = pd.read_csv("analysis/data/forest_landscape_restoration_country_tab.tsv", sep ='\t')
df['retweeted_username'].replace('', np.nan, inplace=True)
df.dropna(subset=['retweeted_username'], inplace=True)
df['entities.mentions'].replace('', np.nan, inplace=True)
df.dropna(subset=['entities.mentions'], inplace=True)
df['entities.mentions'] = df['entities.mentions'].apply(literal_eval)
# print(df.head())
xx = get_dict()
for index, row in df.iterrows():
  orgNode = row["retweeted_username"]
  mym = row["year_month"]
  data = row["entities.mentions"]
  for ele in data:
    destNode = str(ele).replace("@", "")
    mString = orgNode + "\t" + destNode + "\t" + mym + "\n"
    mFile.write(mString)

mFile.close()


<ipython-input-13-ccf2f0f8b12f>:16: DtypeWarning: Columns (29,31,32,33,39,54,55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("analysis/data/forest_landscape_restoration_country_tab.tsv", sep ='\t')


In [ ]:

!python  TSCAN/run.py



/content/gdrive/MyDrive/Yin/landscape/TSCAN/run.py:16: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if method is "1":
/content/gdrive/MyDrive/Yin/landscape/TSCAN/run.py:18: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if method is "2":
/content/gdrive/MyDrive/Yin/landscape/TSCAN/run.py:20: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if method is "3":
Dataset name(str): 
Aborted!
